In [1]:
# from surprise import SVD, Reader, Dataset
# from surprise.model_selection import train_test_split
from sklearn import linear_model
import sklearn
import gzip
import os
from collections import defaultdict
import random
import pandas as pd
from sklearn import linear_model
import dateutil.parser
from datetime import timedelta
from tqdm import tqdm
import csv
import pickle
import numpy as np
import xgboost as xgb

In [11]:
import os
import pickle

In [ ]:

# !pip install implicit
# !pip install surprise
# from implicit import bpr
# from surprise import SVD, Reader, Dataset
# from surprise.model_selection import train_test_split

path = "."
os.chdir(path)

def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

def readCSV(path):
    f = gzip.open(path, 'rt')
    c = csv.reader(f)
    header = next(c)
    for l in c:
        d = dict(zip(header,l))
        yield d['user_id'],d['recipe_id'],d

def parse(f):
    for l in gzip.open(f):
        yield eval(l)

In [12]:
path = "/content/drive/MyDrive/UCSD_assignments/CS258/Assignment2/working_folder"
os.chdir(path)

In [ ]:
#print(user_data[0])
#userName : string
#jobs : nested list [['Tổng công ty IDICO', 'Chuyên viên Kỹ thuật XD', [[1, 1, 1998], [1, 1, 2013], 1], '', '']]
#currentPlace : ['Thành phố Hồ Chí Minh, Việt Nam', [[], 108230990, 1066296640, 1]]
#previousPlaces : [['tp. Nam Định, Nam Định, Việt Nam', [[], 204200000, 1061683330, 1]]]
#education : nested list
#gPlusUserId : string

In [ ]:
#print(places_data[0])
#attributes of places_data
#name : string
#price : None / 
#address : list(strings)
#hours : [['Monday', [['6:30 am--4:15 pm']], ['Tuesday', [['6:30 am--4:15 pm']]]] sometimes number like 1 is also there not sure why
#phone : string
#gPlusPlaceId : string
#gps: [float,float]

In [ ]:
state_list = ["AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA","KS","KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY","NC","ND","OH","OK","OR","PA","RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY"]
state_list = set(state_list)

In [ ]:
'''
req_places = []
for d in places_data:
    t = d["address"][-1].split()
      for ad in t:
        if ad in state_list:
            req_places.append(d)
            '''

'\nreq_places = []\nfor d in places_data:\n    t = d["address"][-1].split()\n      for ad in t:\n        if ad in state_list:\n            req_places.append(d)\n            '

In [20]:
with open("USA_places.pkl","rb") as fp:
    req_places = pickle.load(fp)

In [ ]:
#places_data = list(parse("/content/drive/MyDrive/UCSD_assignments/CS258/Assignment2/places.clean.json.gz"))
#user_data = list(parse("/content/drive/MyDrive/UCSD_assignments/CS258/Assignment2/users.clean.json.gz"))
#review_data = list(parse("/content/drive/MyDrive/UCSD_assignments/CS258/Assignment2/reviews.clean.json.gz"))

print("done")

done


In [ ]:
# review_data = list(parse("/content/drive/MyDrive/UCSD_assignments/CS258/Assignment2/reviews.clean.json.gz"))

In [ ]:
# with open("/content/drive/MyDrive/UCSD_assignments/CS258/Assignment2/review_data_1.pkl","rb") as fp:
#   review_data1 = pickle.load(fp)

# with open("/content/drive/MyDrive/UCSD_assignments/CS258/Assignment2/review_data_2.pkl","rb") as fp:
#   review_data1 = pickle.load(fp)

# with open("/content/drive/MyDrive/UCSD_assignments/CS258/Assignment2/review_data_3.pkl","rb") as fp:
#   review_data1 = pickle.load(fp)

In [ ]:
#review_data1[2000]

NameError: name 'review_data1' is not defined

In [ ]:
req_places[100000]

{'name': 'Office Depot',
 'price': '$$',
 'address': ['1406 W Expy 83', 'Weslaco, TX 78596'],
 'hours': [['Monday', [['8:00 am--9:00 pm']]],
  ['Tuesday', [['8:00 am--9:00 pm']]],
  ['Wednesday', [['8:00 am--9:00 pm']], 1],
  ['Thursday', [['8:00 am--9:00 pm']]],
  ['Friday', [['8:00 am--9:00 pm']]],
  ['Saturday', [['9:00 am--9:00 pm']]],
  ['Sunday', [['10:00 am--7:00 pm']]]],
 'phone': '(956) 968-6207',
 'closed': False,
 'gPlusPlaceId': '103560782244834035321',
 'gps': [26176207.0, -98024823.0]}

In [ ]:
# # with open("USA_places.pkl","wb") as fp:
# #   pickle.dump(req_places,fp)

# with open("USA_places.pkl","rb") as fp:
#     req_places = pickle.load(fp)

In [ ]:
place_ids = [d['gPlusPlaceId'] for d in req_places]
place_id_set = set(place_ids)

In [ ]:
dollar =set([d['price'] for d in req_places])
dollar

{'', '$', '$$', '$$$', None}

In [ ]:
# review1_req = []
# for d in review_data1:
#     if d['gPlusPlaceId'] in place_id_set:
#         review1_req.append(d)

In [ ]:
# with open("USA_reviews1.pkl","wb") as fp:
#   pickle.dump(review1_req,fp)

# with open("USA_reviews2.pkl","wb") as fp:
#   pickle.dump(review1_req,fp)

# with open("USA_reviews3.pkl","wb") as fp:
#   pickle.dump(review1_req,fp)

with open("USA_reviews1.pkl","rb") as fp:
    review1_req = pickle.load(fp)

with open("USA_reviews2.pkl","rb") as fp:
    review2_req = pickle.load(fp)

with open("USA_reviews3.pkl","rb") as fp:
    review3_req = pickle.load(fp)

In [ ]:
review1_req.extend(review2_req)
review1_req.extend(review3_req)

In [ ]:
review1_req_indices = np.random.choice(len(review1_req), int(0.1*len(review1_req)))
review1_req_set = set(review1_req_indices)
review1_req_train = []
review1_req_test = []
for i in range(len(review1_req)):
    if i not in review1_req_set:
        review1_req_train.append(review1_req[i])
    else:
        review1_req_test.append(review1_req[i])

In [10]:
# with open("USA_reviews_train.pkl","wb") as fp:
#     pickle.dump(review1_req_train,fp)
    
# with open("USA_reviews_test.pkl","wb") as fp:
#     pickle.dump(review1_req_test,fp)

with open("USA_reviews_train.pkl","rb") as fp:
    review1_req_train = pickle.load(fp)
    
with open("USA_reviews_test.pkl","rb") as fp:
    review1_req_test = pickle.load(fp)

In [5]:
with open("Places_GPS_mapping.pkl","rb") as fp:
    place_id_co_ordinates_mapping = pickle.load(fp)

place_id_set = set(place_id_co_ordinates_mapping.keys())

In [6]:
# with open("Places_ids_incorrect_gps.pkl","rb") as fp:
#     place_id_set_incorrect = pickle.load(fp)

In [11]:
filtered_train = []
filtered_test = []

# for d in review1_req_train:
#   if d['gPlusPlaceId'] in place_id_set:
#     d['gps'] = place_id_co_ordinates_mapping[d['gPlusPlaceId']]
#     filtered_train.append(d)

for d in review1_req_test:
  if d['gPlusPlaceId'] in place_id_set:
    d['gps'] = place_id_co_ordinates_mapping[d['gPlusPlaceId']]
    filtered_test.append(d)


In [12]:
print(len(review1_req_train))
print(len(filtered_train))
print()
print(len(review1_req_test))
print(len(filtered_test))

4989620
4911822

524541
516501


In [13]:
# with open("Filtered_train.pkl","wb") as fp:
#     pickle.dump(filtered_train,fp)

with open("Filtered_test.pkl","wb") as fp:
    pickle.dump(filtered_test,fp)

In [ ]:
#filtering user data
# user_ids = set([d['gPlusUserId'] for d in review3_req])

In [ ]:
# with open("review1_uids.pkl","rb") as fp:
#     user_ids1 = pickle.load(fp)
# with open("review2_uids.pkl","rb") as fp:
#     user_ids2 = pickle.load(fp)
# with open("review3_uids.pkl","rb") as fp:
#     user_ids3 = pickle.load(fp)

In [ ]:
# all_user_ids = user_ids1.union(user_ids2).union(user_ids3)
with open("All_req_uids.pkl","rb") as fp:
    all_user_ids = pickle.load(fp)

In [ ]:
# req_users_info = []
# for d in user_data:
#     if d['gPlusUserId'] in all_user_ids:
#         req_users_info.append(d)

In [ ]:
# with open("User_Info_req.pkl","wb") as fp:
#     pickle.dump(req_users_info,fp)
with open("User_Info_req.pkl","rb") as fp:
    req_users = pickle.load(fp)


In [ ]:
data = []
recipe_info = defaultdict(dict)

# Utility data structures to stores Users per Item and Items Per User
usersPerRecipe = defaultdict(set)
recipesPerUser = defaultdict(set)

# Utility data structures to stores Ratings per Item and Ratings Per User
userRatings = defaultdict(dict)
recipeRatings = defaultdict(dict)

for user,recipe,rating in readCSV("trainInteractions.csv.gz"):
    data.append({'user': user, 'recipe': recipe, 'rating':rating})
    
training_set_size = 400000
train = data[:training_set_size]
valid = data[training_set_size:]

for d in train:
    userRatings[d['user']][d['recipe']] = int(d['rating'])
    recipeRatings[d['recipe']][d['user']] = int(d['rating'])

In [ ]:
review3_req[0]

{'rating': 5.0,
 'reviewerName': 'Hans Katraz',
 'reviewText': 'This is a mini Seaworld,but more up close and personal without the crowds.We talked for many many years about stopping here and we should have done it a long time ago.Great price and neat shows. Great job !!!',
 'categories': ['Aquarium', 'Gift Shop', 'Theme Park'],
 'gPlusPlaceId': '113457001384897145613',
 'unixReviewTime': 1341630203,
 'reviewTime': 'Jul 6, 2012',
 'gPlusUserId': '112232251331336783091'}

In [ ]:
req_users[1]

{'userName': 'Jacquelyn Dorris',
 'jobs': [['PS Medical Supplies, Inc.',
   'Customer Service',
   [[1, 1, 2012], [1, 1, 2013], 1],
   '',
   '']],
 'currentPlace': ['Pomona, CA', [[], 340552270, -1177523050, 1]],
 'previousPlaces': [['Upland, Ca', [[], 340975100, -1176483880, 1]],
  ['Azusa, CA', [[], 341336190, -1179075630, 1]],
  ['Rancho Cucamonga, CA', [[], 341063990, -1175931080, 1]]],
 'education': [[[], [], [], [], [], 6],
  [['Upland High School', '', [[1, 1, 2008], [1, 1, 2012]], '', '']]],
 'gPlusUserId': '100000035085750632094'}

In [ ]:
data = [[d['gPlusUserId'],d['gPlusPlaceId'], d['rating']] for d in review3_req ]

In [ ]:
data[0]

['112232251331336783091', '113457001384897145613', 5.0]

In [ ]:
train = data[:int(len(data)*0.8)]

In [ ]:
df = pd.DataFrame(data, columns=['user', 'place','rating'])

reader = Reader(rating_scale=(0, 5))
training_data = Dataset.load_from_df(df[['user', 'place', 'rating']], reader)

trainset, validationset = train_test_split(training_data, shuffle = False, test_size = 0.20)

In [ ]:
model = SVD(n_epochs= 15, lr_all= 0.005, n_factors= 1, reg_all= 2)    
model.fit(trainset)

In [ ]:
predictions_train = model.test(train)
predictions_val = model.test(validationset)

In [ ]:
sse = 0
for p in predictions_train:
    sse += (p.r_ui - p.est)**2

print('Training set MSE is equal to ', sse / len(predictions_train))

sse = 0
for p in predictions_val:
    sse += (p.r_ui - p.est)**2

print('Validation set MSE is equal to ', sse / len(predictions_val))

Training set MSE is equal to  1.1906850944018155
Validation set MSE is equal to  1.540744962949999


In [19]:
req_places[100000]

NameError: ignored

In [ ]:
req_places[15]

{'address': ['7130 Airport Hwy', 'Ste 5', 'Holland, OH 43528'],
 'closed': True,
 'gPlusPlaceId': '118117723948195890487',
 'gps': [41.610549, -83.712633],
 'hours': [['Monday', [['11:00 am--11:00 pm']]],
  ['Tuesday', [['11:00 am--11:00 pm']]],
  ['Wednesday', [['11:00 am--11:00 pm']]],
  ['Thursday', [['11:00 am--11:00 pm']]],
  ['Friday', [['11:00 am--1:00 am']]],
  ['Saturday', [['11:00 am--1:00 am']], 1],
  ['Sunday', [['12:00--11:00 pm']]]],
 'name': "Hungry Howie's Pizza & Subs",
 'phone': '(419) 867-9991',
 'price': None}

In [ ]:
gps_locs_set = set()
place_id_set_incorrect = set()
count_gps_places_none = 0

for i in range(len(req_places)):
  if req_places[i]['gps']:
    gps_locs_set.add(tuple(req_places[i]['gps']))
    if abs(tuple(req_places[i]['gps'])[0]) > 90 or abs(tuple(req_places[i]['gps'])[1]) > 180:
      place_id_set_incorrect.add(req_places[i]['gPlusPlaceId'])
  else:
    count_gps_places_none+=1


# print("Incorrect GPS co-ord for places:",len(place_id_set_incorrect))
# print("Correct GPS co-ord for places:",len(req_places)-len(place_id_set_incorrect) - count_gps_places_none)
# print("GPS co-ord for places missing:",count_gps_places_none)

In [16]:
print("Incorrect GPS co-ord for places:",len(place_id_set_incorrect))
print("Correct GPS co-ord for places:",len(req_places)-len(place_id_set_incorrect) - count_gps_places_none)
print("GPS co-ord for places missing:",count_gps_places_none)

Incorrect GPS co-ord for places: 40089
Correct GPS co-ord for places: 1242392
GPS co-ord for places missing: 18845


In [14]:
from collections import defaultdict
place_id_co_ordinates_mapping = defaultdict()

for i in range(len(req_places)):
  if req_places[i]['gps']:
    if abs(tuple(req_places[i]['gps'])[0]) <= 90 and abs(tuple(req_places[i]['gps'])[1]) <= 180:
    #if req_places[i]['gPlusPlaceId'] not in place_id_set_incorrect:
      place_id_co_ordinates_mapping[req_places[i]['gPlusPlaceId']] = tuple(req_places[i]['gps'])



In [17]:
# with open("Places_GPS_mapping.pkl","wb") as fp:
#     pickle.dump(place_id_co_ordinates_mapping,fp)

In [10]:
# with open("Places_ids_incorrect_gps.pkl","wb") as fp:
#     pickle.dump(place_id_set_incorrect,fp)

In [ ]:
count = 0
for d in req_users:
  if d['currentPlace']:
    

1836152

In [ ]:
data = [[d['gPlusUserId'],d['gPlusPlaceId'], d['rating']] for d in review1_req ]

In [ ]:
df = pd.DataFrame(data, columns=['user', 'place','rating'])


In [ ]:
import matplotlib.pyplot as plt

#ax = df[['rating']].plot(kind='bar', title ="Rating Distribution", figsize=(15, 10), legend=True, fontsize=12)
ax = df[['rating']].plot(kind='bar', title ="Rating Distribution",legend=True)
ax.set_xlabel("Rating", fontsize=12)
ax.set_ylabel("Rating count", fontsize=12)
plt.show()

In [ ]:
df[['rating']].value_counts()

rating
5.0       2986312
4.0       1105932
3.0        550585
2.0        479896
1.0        391401
0.0            35
dtype: int64

In [15]:
review1_req[12]

NameError: ignored

In [21]:
# with open("Places_GPS_mapping.pkl","rb") as fp:
#     place_id_co_ordinates_mapping = pickle.load(fp)
# ids = set(place_id_co_ordinates_mapping.keys())
# filtered_places = []

# for d in req_places:
#   if d['gPlusPlaceId'] in ids:
#     filtered_places.append(d)





In [23]:
# with open("Filtered_places.pkl","wb") as fp:
#      pickle.dump(filtered_places,fp)

In [13]:
with open("Filtered_places.pkl","rb") as fp:
     filtered_places = pickle.load(fp)

In [5]:
filtered_places[1000]

{'address': ['13545 Apple Ave', 'Ravenna, MI 49451'],
 'closed': False,
 'gPlusPlaceId': '114122107835358203256',
 'gps': [43.235341, -85.908661],
 'hours': None,
 'name': 'Moss Ridge Golf Club',
 'phone': '(231) 853-5665',
 'price': None}

In [14]:
price_filtered_places = []
for d in filtered_places:
  if d['price'] and len(d['price']):
    price_filtered_places.append(d)


In [15]:
with open("Price_filtered_places.pkl","wb") as fp:
     pickle.dump(price_filtered_places,fp)

In [9]:
with open("Filtered_train.pkl","rb") as fp:
    review_train = pickle.load(fp)

In [7]:
len(price_filtered_places)

price_filtered_place_ids = set([x['gPlusPlaceId'] for x in price_filtered_places])


In [10]:
price_filtered_review_train = []
for d in review_train:
  if d['gPlusPlaceId'] in price_filtered_place_ids:
    price_filtered_review_train.append(d)


In [12]:
import pandas as pd
temp = pd.DataFrame(price_filtered_review_train)

In [14]:
df = temp['categories'].value_counts()

In [20]:
cats = df.to_frame()

In [22]:
top_categories = [df.index[i] for i in range(50)]

In [26]:
# with open("Categories_freq.pkl","wb") as fp:
#      pickle.dump(df,fp)

# with open("Top_Categories_50.pkl","wb") as fp:
#      pickle.dump(top_categories,fp)

# with open("Price_filtered_review_train.pkl","wb") as fp:
#      pickle.dump(price_filtered_review_train,fp)

In [34]:
top_categories = set([tuple(x) for x in top_categories])
cat_filtered_review_train = []

for d in price_filtered_review_train:
  if d['categories'] and tuple(d['categories']) in top_categories:
    cat_filtered_review_train.append(d)


In [36]:
# with open("Cat_price_filtered_review_train.pkl","wb") as fp:
#      pickle.dump(cat_filtered_review_train,fp)

In [16]:
with open("Filtered_test.pkl","rb") as fp:
    review_test = pickle.load(fp)

In [17]:
with open("Top_Categories_50.pkl","rb") as fp:
     top_categories = pickle.load(fp)

In [18]:
price_filtered_places = []
for d in filtered_places:
  if d['price'] and len(d['price']):
    price_filtered_places.append(d)

In [19]:
price_filtered_review_test = []
for d in review_test:
  if d['gPlusPlaceId'] in price_filtered_place_ids:
    price_filtered_review_test.append(d)

In [20]:
len(price_filtered_review_test)

203242

In [21]:
cat_filtered_review_test = []
for d in price_filtered_review_test:
  if d['categories'] and tuple(d['categories']) in top_categories:
    cat_filtered_review_test.append(d)


In [22]:
len(cat_filtered_review_test)

0

In [10]:
with open("Cat_price_filtered_review_test.pkl","wb") as fp:
     pickle.dump(cat_filtered_review_test,fp)